# fkpt: obtain LS values for functions A, Ap, KR1, KR1p

EdS values are A_LS, Ap_LS, KR1_LS, KR1p_LS = 1,0,1,1.   

LCDM values are close to EdS. For MG theories that reduce to LCDM at large scales, the values are the same as in LCDM 


EdS conditions are assumed at early times

In [2]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

# Linear growth equation is

$D''(k,\eta) + (2 + H'/H) D'(k,\eta) - \frac{3}{2} \Omega_m(\eta) \mu(k,\eta) D(\eta) =0$ 

with $\eta=\log(a)$


In [4]:


## Change mu for your MG
def mu(eta, k, OmegaM):
    beta2 = 1.0 / 6.0
    nHS = 1.0
    invH0 = 2997.92458

    mass = (
        (1.0 / invH0)
        * np.sqrt(0.5)
        * (OmegaM * np.exp(-3.0 * eta) + 4.0 * (1.0 - OmegaM)) ** ((2.0 + nHS) / 2.0)
        / (OmegaM + 4.0 * (1.0 - OmegaM)) ** ((1.0 + nHS) / 2.0)
    )

    return 1.0 + (2.0 * beta2 * k**2) / (k**2 + np.exp(2.0 * eta) * mass**2)

#Change f1 and f2 is your background is not LCDM:
 
# f2 =  (2 + H'/H)
def f2(eta, OmegaM):
    return 2.0 - 3.0 / (2.0 * (1.0 + (1.0 - OmegaM) / OmegaM * np.exp(3.0 * eta)))

# f1 =  3/2 OmegaM(eta)
def f1(eta, OmegaM):
    return 3.0 / (2.0 * (1.0 + (1.0 - OmegaM) / OmegaM * np.exp(3.0 * eta)))




In [16]:
def fkpt_LS_values(z,OmM,f0,
    mu_func,
    f1_func,
    f2_func
):
    
    if z<0.00001*1.01:
        z=0.00001*1.01
    output = compute_LSvalues(
        etaev=np.log(1/(1+z)),
        xev=0.0,kev=1e-8,pev=1e-8,
        f0ev=f0,OmegaM=OmM,
        mu_func=mu_func,
        f1_func=f1_func,
        f2_func=f2_func,
    )
    return output



def compute_LSvalues(
    etaev,
    xev,
    kev,
    pev,
    f0ev,
    OmegaM,
    mu_func,
    f1_func,
    f2_func,
    *,
    etainI=-4.0,
    etafinal=0.0,
    rtol=1e-6,
    atol=1e-9,
):
    import numpy as np
    from scipy.integrate import solve_ivp

    # --------------------------------------------------
    # Precompute geometry (constants!)
    # --------------------------------------------------

    kp  = np.sqrt(kev*kev + pev*pev + 2.0*kev*pev*xev)
    kpm = np.sqrt(kev*kev + pev*pev - 2.0*kev*pev*xev)

    pk_ratio2 = (pev / kev)**2
    geom_p = 1.0 + pk_ratio2 + 2.0 * pev/kev * xev
    geom_m = 1.0 + pk_ratio2 - 2.0 * pev/kev * xev

    one_minus_x2 = 1.0 - xev*xev

    # --------------------------------------------------
    # ODE system (optimized RHS)
    # --------------------------------------------------

    def rhs(eta, y):
        (
            D3, dD3,
            D2p, dD2p,
            D2m, dD2m,
            Dpk, dDpk,
            Dpp, dDpp
        ) = y

        # Cache expensive background functions
        f1v = f1_func(eta, OmegaM)
        f2v = f2_func(eta, OmegaM)

        mu_k  = mu_func(eta, kev, OmegaM)
        mu_p  = mu_func(eta, pev, OmegaM)
        mu_kp = mu_func(eta, kp,  OmegaM)
        mu_km = mu_func(eta, kpm, OmegaM)
        
        

        # sourceD2 (x and -x)
        SD2_p = (
            f1v * mu_kp
            - f1v * (mu_k + mu_p - mu_kp) * xev*xev
            + f1v * (
                mu_kp * (pev/kev + kev/pev) * xev
                - kev*xev/pev * mu_k
                - pev*xev/kev * mu_p
            )
        )

        SD2_m = (
            f1v * mu_km
            - f1v * (mu_k + mu_p - mu_km) * xev*xev
            - f1v * (
                mu_km * (pev/kev + kev/pev) * xev
                - kev*xev/pev * mu_k
                - pev*xev/kev * mu_p
            )
        )

        # Source 3I
        S3I = (
            (f1v * (mu_p + mu_kp - mu_k) * D2p * Dpp
             + SD2_p * Dpk * Dpp * Dpp)
            * one_minus_x2 / geom_p
        )

        S3I += (
            (f1v * (mu_p + mu_km - mu_k) * D2m * Dpp
             + SD2_m * Dpk * Dpp * Dpp)
            * one_minus_x2 / geom_m
        )

        # Source 3II (symmetrized)
        S3II = (
            -f1v * (mu_p + mu_kp - 2.0*mu_k)
            * Dpp * (D2p + Dpk*Dpp*xev*xev)
            -f1v * (mu_kp - mu_k)
            * Dpk * Dpp * Dpp
        )
        S3II *= 2.0

        return np.array([
            dD3,
            -f2v*dD3 + f1v*mu_k*D3 + S3I + S3II,

            dD2p,
            -f2v*dD2p + f1v*mu_kp*D2p + SD2_p*Dpk*Dpp,

            dD2m,
            -f2v*dD2m + f1v*mu_km*D2m + SD2_m*Dpk*Dpp,

            dDpk,
            -f2v*dDpk + f1v*mu_k*Dpk,

            dDpp,
            -f2v*dDpp + f1v*mu_p*Dpp,
        ])

    # --------------------------------------------------
    # Initial conditions
    # --------------------------------------------------

    Dplusi  = np.exp(etainI)
    D2plusi = 3.0*np.exp(2.0*etainI)/7.0 * one_minus_x2

    D3symmi = (
        5.0/7.0*np.exp(3.0*etainI)/9.0
        * (
            one_minus_x2 / geom_p
            + one_minus_x2 / geom_m
        )
        * one_minus_x2
    )

    y0 = np.array([
        D3symmi, 3.0*D3symmi,
        D2plusi, 2.0*D2plusi,
        D2plusi, 2.0*D2plusi,
        Dplusi,  Dplusi,
        Dplusi,  Dplusi
    ])

    # --------------------------------------------------
    # Solve ODE
    # --------------------------------------------------

    sol = solve_ivp(
        rhs,
        (etainI, etafinal),
        y0,
        rtol=rtol,
        atol=atol,
        method="RK45",
        dense_output=True
    )

    Y = sol.sol(etaev)

    D3, dD3, D2p, dD2p, _, _, Dpk, dDpk, Dpp, dDpp = Y

    # --------------------------------------------------
    # Observables (NO interpolation!)
    # --------------------------------------------------

    C  = (3.0/7.0) * Dpk * Dpp
    Cp = (3.0/7.0) * (dDpk*Dpp + Dpk*dDpp)

    ALS      = D2p / C
    AprimeLS = dD2p / C - D2p * Cp / (C*C)

    KR1LS  = D3  / (Dpk*Dpp*Dpp) * 21.0/5.0
    KR1pLS = dD3 / (Dpk*Dpp*Dpp) * 21.0/5.0 / (3.0*f0ev)

    return np.array([ALS, AprimeLS, KR1LS, KR1pLS])


In [18]:
%%time
# WARNING: A change on z or OmM implies a change on f0. Hence, be careful, or you will obtain non-sense KR1p_LS values
fkpt_LS_values(z=0.5,OmM=0.3,f0=0.749238,
    mu_func=mu,
    f1_func=f1,
    f2_func=f2
)

CPU times: user 12.4 ms, sys: 2.2 ms, total: 14.6 ms
Wall time: 12.8 ms


array([1.00374172, 0.00884026, 1.00699984, 1.01630063])